In [33]:
import requests
import json
from bs4 import BeautifulSoup

In [34]:
def get_all_article_links():
    base = "https://ilos.com.br/categoria/insights/"
    page = 1
    links = set()

    while True:
        url = base if page == 1 else f"{base}page/{page}/"
        print(f"Coletando página {page}: {url}")

        r = requests.get(url, timeout=20)

        if r.status_code != 200:
            print("Fim da paginação (status != 200)")
            break

        soup = BeautifulSoup(r.text, "html.parser")

        page_links = soup.select(
            'a[data-wpel-link="internal"][title^="View Post"]'
        )

        if not page_links:
            print(f"⚠️ Página {page} sem artigos (HTML mudou?)")
            break

        before = len(links)

        for a in page_links:
            links.add(a["href"])

        after = len(links)
        print(f" +{after - before} artigos")

        page += 1

    return list(links)


In [35]:
def scrape_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.find("h1", class_="uicore-title")
    meta = soup.find("div", class_="uicore-entry-meta")
    article = soup.find("article")

    # Autor
    author = None
    if meta:
        a = meta.select_one('a[href*="/author/"]')
        if a:
            author = a.get_text(strip=True)

    # Data
    date = None
    if meta:
        d = meta.find("span", class_="ui-blog-date")
        if d:
            date = d.get_text(strip=True)

    # Categoria
    category = None
    if meta:
        c = meta.select_one(".uicore-post-category a")
        if c:
            category = c.get_text(strip=True)

    # Conteúdo HTML
    content_html = str(article) if article else None

    # JSON-LD
    tags = []
    thumbnail = None

    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string)
            graph = data.get("@graph", [data])

            for node in graph:
                if node.get("@type") == "Article":
                    thumbnail = node.get("thumbnailUrl")

                    k = node.get("keywords")
                    if isinstance(k, str):
                        k = [k]
                    tags.extend(k or [])
        except:
            continue

    return {
        "title": title.get_text(strip=True) if title else None,
        "author": author,
        "date": date,
        "thumbnail": thumbnail,
        "tags": tags,
        "category": category,
        "content_html": content_html,
        "source_url": url
    }


In [36]:
articles = []

links = get_all_article_links()
print(f"{len(links)} artigos encontrados")

for i, link in enumerate(links, 1):
    try:
        print(f"[{i}/{len(links)}] {link}")
        articles.append(scrape_article(link))
    except Exception as e:
        print("Erro:", link, e)

with open("insights.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)


Coletando página 1: https://ilos.com.br/categoria/insights/
 +12 artigos
Coletando página 2: https://ilos.com.br/categoria/insights/page/2/
 +12 artigos
Coletando página 3: https://ilos.com.br/categoria/insights/page/3/
 +12 artigos
Coletando página 4: https://ilos.com.br/categoria/insights/page/4/
 +12 artigos
Coletando página 5: https://ilos.com.br/categoria/insights/page/5/
 +12 artigos
Coletando página 6: https://ilos.com.br/categoria/insights/page/6/
 +12 artigos
Coletando página 7: https://ilos.com.br/categoria/insights/page/7/
 +12 artigos
Coletando página 8: https://ilos.com.br/categoria/insights/page/8/
 +12 artigos
Coletando página 9: https://ilos.com.br/categoria/insights/page/9/
 +12 artigos
Coletando página 10: https://ilos.com.br/categoria/insights/page/10/
 +12 artigos
Coletando página 11: https://ilos.com.br/categoria/insights/page/11/
 +12 artigos
Coletando página 12: https://ilos.com.br/categoria/insights/page/12/
 +12 artigos
Coletando página 13: https://ilos.com.br/

In [ ]:
# Montagem do Json final com a estrutura desejada
# Não funciona mais
data = {
    "title": title.get_text(strip=True) if title else None,
    "author": author,
    "date": date,
    "tags": tags,
    "thumbnail_url": thumbnail,
    "category": category,
    "content_html": str(article) if article else None,
    "source_url": url
}

with open("pagina2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

''''
# Se quiser, ainda pode salvar o HTML para inspeção visual
with open("pagina2.html", "w", encoding="utf-8") as f:
    f.write(str(article) if article else "")'''


'\'\n# Se quiser, ainda pode salvar o HTML para inspeção visual\nwith open("pagina2.html", "w", encoding="utf-8") as f:\n    f.write(str(article) if article else "")'

In [ ]:

# Não está em uso(Alterar parametros)
title_text = title.get_text(strip=True) if title else ""
author_text = author or ""
date_text = date or ""
category_text = category or ""

# Monta o topo da página
header_html = f"""
<h1>{title_text}</h1>
<p style="color:#555;">
  {author_text} • {date_text} • {category_text}
</p>
<hr>
"""

# Corpo do artigo
article_html = str(article) if article else ""

# Junta tudo
pagina2_html = header_html + article_html
with open("pagina2_completa.html", "w", encoding="utf-8") as f:
    f.write(pagina2_html)